In [1]:
from sklearn.datasets import load_iris
import pandas as pd

#아이리스 데이터 로드
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
print(iris_df.head())
iris_df['species'] = iris.target #품종이 타겟(0:Setosa, 1:Versicolor, 2:Virginica)
print(iris_df.head())
#print(iris_DESOR) #print(iris_df.head()) #print(iris_df.info()) #print(iris_df.descibe())
input_data = iris_df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']].to_numpy()
target_data = iris_df[['species']].to_numpy()
print(input_data[:5])
print(target_data[:5])

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   species  
0        0  
1        0  
2        0  
3        0  
4   

In [2]:
from sklearn.model_selection import train_test_split

# 훈련 데이터와 테스트 데이터 나누기
train_input, test_input, train_target, test_target = train_test_split(
    input_data, target_data, random_state=42
)

print(train_input.shape, test_input.shape)
print(train_target.shape, test_target.shape)

(112, 4) (38, 4)
(112, 1) (38, 1)


In [3]:
from sklearn.preprocessing import StandardScaler

# 데이터 표준화
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [4]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target.ravel())

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.9821428571428571
1.0


In [5]:
print(lr.predict(test_scaled[:5]))

[1 0 2 1 1]


In [6]:
print(lr.classes_)       # 클래스(품종) 순서
print(lr.coef_.shape, lr.intercept_.shape)  # 계수와 절편의 shape

[0 1 2]
(3, 4) (3,)


In [7]:
import numpy as np

#로지스틱 회귀의 다중분류에서는 소프트맥스 함수를 이용함
#소프트맥스의 계산 과정을 단계별로 확인

# 1단계: decision_function으로 z값(선형결합 결과) 계산
decision = lr.decision_function(test_scaled[:5])
print("z값\n", np.round(decision, 2))

# 2단계: z값에 e^z 적용
ez = np.exp(decision)
print("\ne^z\n", np.round(ez, 2))

# 3단계: e^z의 합 계산
esum = np.sum(ez, axis=1, keepdims=True)
print("\ne^z의 합\n", np.round(esum, 2))

# 4단계: softmax 확률 계산
s = ez / esum
print("\nsoftmax 결과(확률)\n", np.round(s, 3))
print("\nsoftmax 합(검증):", np.sum(s, axis=1))

z값
 [[ -3.87   4.24  -0.37]
 [ 13.03   7.9  -20.94]
 [-18.82   2.65  16.17]
 [ -4.19   3.64   0.56]
 [ -6.65   4.91   1.75]]

e^z
 [[2.00000000e-02 6.95300000e+01 6.90000000e-01]
 [4.57513460e+05 2.70899000e+03 0.00000000e+00]
 [0.00000000e+00 1.41400000e+01 1.04882609e+07]
 [2.00000000e-02 3.79100000e+01 1.75000000e+00]
 [0.00000000e+00 1.35470000e+02 5.73000000e+00]]

e^z의 합
 [[7.0240000e+01]
 [4.6022245e+05]
 [1.0488275e+07]
 [3.9680000e+01]
 [1.4120000e+02]]

softmax 결과(확률)
 [[0.    0.99  0.01 ]
 [0.994 0.006 0.   ]
 [0.    0.    1.   ]
 [0.    0.956 0.044]
 [0.    0.959 0.041]]

softmax 합(검증): [1. 1. 1. 1. 1.]


In [8]:
from scipy.special import softmax

proba_scipy = softmax(decision, axis=1)
print(np.round(proba_scipy, decimals=3))
print("softmax 합(검증):", np.sum(proba_scipy, axis=1))

[[0.    0.99  0.01 ]
 [0.994 0.006 0.   ]
 [0.    0.    1.   ]
 [0.    0.956 0.044]
 [0.    0.959 0.041]]
softmax 합(검증): [1. 1. 1. 1. 1.]
